In [8]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

dataSource = 'participant' #'pilot', 'presented'

path = './data/interaction_ros_logs'

df = pd.read_csv(os.path.join(path,'{}_durations.csv'.format(dataSource)))
display(df)

,PID,Trial,Duration-1,Duration-2,Duration-3,Duration-4,Duration-5,Duration-6,Duration-7,Duration-8,Duration-9,Duration-10,Duration-11,Duration-12,totalNeglect,totalInteract,skipFirstNeglect,skipFirstInteract
0,1,1,3.5316,13.93,32.59,4.13,27.61,5.61,34.61,7.49,24.10,6.03,22.09,8.11,144.531600,45.300000,141.00,31.370000
1,1,2,3.5062,2.11,32.61,7.51,27.60,5.99,34.62,5.51,24.10,3.89,22.09,1.00,144.526200,26.010000,141.02,23.900000
2,1,3,3.5200,2.14,32.62,4.65,27.59,6.55,34.61,5.01,24.09,3.01,22.09,1.32,144.520000,22.680000,141.00,20.540000
3,2,1,3.5300,11.64,32.61,14.60,27.61,11.00,34.60,13.35,24.10,9.96,22.10,22.21,144.550000,82.760000,141.02,71.120000
4,2,2,3.5400,34.05,32.60,7.90,27.60,7.74,34.63,17.12,24.10,5.38,22.09,4.58,144.560000,76.770000,141.02,42.720000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,34,2,3.5300,7.18,32.63,14.27,27.61,9.95,34.61,15.61,24.11,11.56,22.12,7.25,144.610000,65.820000,141.08,58.640000
101,34,3,3.5387,11.26,32.62,5.68,27.61,3.17,34.62,3.13,24.11,1.18,22.11,5.12,144.608701,29.539999,141.07,18.279999
102,35,1,3.5100,4.70,32.62,7.14,27.62,11.96,34.63,12.16,24.12,23.81,22.10,8.39,144.600000,68.160000,141.09,63.460000
103,35,2,3.5200,2.60,32.63,5.05,27.61,11.56,34.64,8.24,24.12,4.08,22.13,6.51,144.650000,38.040000,141.13,35.440000


In [9]:
df.keys()

Index(['PID', 'Trial', 'Duration-1', 'Duration-2', 'Duration-3', 'Duration-4',
       'Duration-5', 'Duration-6', 'Duration-7', 'Duration-8', 'Duration-9',
       'Duration-10', 'Duration-11', 'Duration-12', 'totalNeglect',
       'totalInteract', 'skipFirstNeglect', 'skipFirstInteract'],
      dtype='object')

In [10]:
durations = df.loc[:, ['Duration-{}'.format(i) for i in range(1,13)]].to_numpy()


rad_table = np.zeros((durations.shape[0],int(durations.shape[1]/2)+2))

# col 0 is total rad
totalInteract = df.loc[:, ['totalInteract']].to_numpy()
totalNeglect = df.loc[:, ['totalNeglect']].to_numpy()
temp = np.true_divide(totalInteract, np.add(totalNeglect, totalInteract))
rad_table[:,0] = np.reshape(temp, (temp.shape[0],))

# col 1 is skip 1 rad
skipFirstInteract = df.loc[:, ['skipFirstInteract']].to_numpy()
skipFirstNeglect = df.loc[:, ['skipFirstNeglect']].to_numpy()
temp = np.true_divide(skipFirstInteract, np.add(skipFirstNeglect, skipFirstInteract))
rad_table[:,1] = np.reshape(temp, (temp.shape[0],))

# col 2-8 are step rad
# evens [0,2,4,...] (duration-1, duration-3, duration-5) neglect time
# odds [1,3,5,...] (duration-2, duration-4, duration-6) interaction time

neglect = durations[:,0:13:2]
interact = durations[:, 1:13:2]

for i in range(0,6):
    rad_table[:,i+2] = np.true_divide(interact[:,i], np.add(interact[:,i], neglect[:,i]))
    
    

rdf = pd.DataFrame(data=rad_table, columns=['Total','skipFirst']+['Step-{}'.format(i+1) for i in range(0,6)])
idf = df.loc[:, ['PID','Trial']]

result = pd.concat([idf,rdf], axis=1)
display(result)

,PID,Trial,Total,skipFirst,Step-1,Step-2,Step-3,Step-4,Step-5,Step-6
0,1,1,0.238633,0.181992,0.797750,0.112473,0.168874,0.177910,0.200133,0.268543
1,1,2,0.152519,0.144919,0.375699,0.187188,0.178327,0.137304,0.138978,0.043309
2,1,3,0.135646,0.127151,0.378092,0.124765,0.191857,0.126451,0.111070,0.056386
3,2,1,0.364084,0.335250,0.767304,0.309257,0.284900,0.278415,0.292425,0.501241
4,2,2,0.346858,0.232502,0.905826,0.195062,0.219015,0.330821,0.182497,0.171729
...,...,...,...,...,...,...,...,...,...,...
100,34,2,0.312788,0.293611,0.670401,0.304264,0.264909,0.310832,0.324082,0.246851
101,34,3,0.169625,0.114716,0.760878,0.148303,0.102989,0.082914,0.046659,0.188028
102,35,1,0.320361,0.310242,0.572473,0.179577,0.302173,0.259885,0.496766,0.275172
103,35,2,0.208222,0.200714,0.424837,0.134023,0.295124,0.192164,0.144681,0.227304


In [11]:
result.to_csv(os.path.join(path, '{}_rad_table.csv'.format(dataSource)), index=False, header=True)